In [1]:
import numpy as np
import matplotlib.pyplot as plt
# timing via time()
from time import time
from matplotlib import cm
import copy
%matplotlib inline

In [2]:
def draw_image(mat, cmap='inferno', powern=0.5, dpi=72):
    ## Value normalization
    # Apply power normalization, because number of iteration is 
    # distributed according to a power law (fewer pixels have 
    # higher iteration number)
    mat = np.power(mat, powern)
    
    # Colormap: set the color the black for values under vmin (inner points of
    # the set), vmin will be set in the imshow function
    # new_cmap = copy.copy(cm.get_cmap(cmap))  # depricated
    # new_cmap = copy.copy(cm._colormaps[cmap])    # GH
    new_cmap = copy.copy(plt.get_cmap())
  
    new_cmap.set_under('black')
    
    ## Plotting image
    
    # Figure size
    plt.figure(figsize=(mat.shape[0]/dpi, mat.shape[1]/dpi))
    
    # Plotting mat with cmap
    # vmin=1 because smooth iteration count is always > 1
    # We need to transpose mat because images use row-major
    # ordering (C convention)
    # origin='lower' because mat[0,0] is the lower left pixel
    plt.imshow(mat.T, cmap=new_cmap, vmin=1, origin = 'lower')
    
    # Remove axis and margins
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.axis('off')


In [3]:
from numba import prange, jit, njit

In [12]:
# counts the number of iterations until the function diverges or
# returns the iteration threshold that we check until
@jit(nopython=True)
def countIterationsUntilDivergent(c, threshold):
    z = complex(0, 0)
    for iteration in range(threshold):
        z = (z*z) + c

        # if abs(z) > 4:                        # 0.063 sec
        if z.real*z.real+z.imag*z.imag > 4*4:   # 0.017 sec
            break
    
    return iteration

The function mandelbrot() is used in three modes: 
* pure python: three comment lines
* jit-compiler: active  `@jit(nopython=True)`  and use   `for x in range`
* njit-OpenMP-compiler: activate `@njit(parallel=True)`and use `for x in prange`

In [5]:
# takes the iteration limit before declaring function as convergent and
# takes the density of the atlas
# create atlas, plot mandelbrot set, display set
#@jit(nopython=True)
@njit(parallel=True)
def mandelbrot(threshold, density):
    # location and size of the atlas rectangle
    realAxis = np.linspace(-0.22, -0.219, density)
    imaginaryAxis = np.linspace(-0.70, -0.699, density)
    realAxisLen = len(realAxis)
    imaginaryAxisLen = len(imaginaryAxis)
    # print(realAxis.dtype)
    # 2-D array to represent mandelbrot atlas
    atlas = np.empty((realAxisLen, imaginaryAxisLen))

    # color each point in the atlas depending on the iteration count
    #for ix in range(realAxisLen):    
    for ix in prange(realAxisLen):
        for iy in range(imaginaryAxisLen):
            cx = realAxis[ix]
            cy = imaginaryAxis[iy]
            c = complex(cx, cy)

            atlas[ix, iy] = countIterationsUntilDivergent(c, threshold)
    
    return atlas

In [6]:
# Parameters
xpixels = 1000
ypixels = xpixels

maxiter = 120
mat = np.zeros((xpixels, ypixels))

In [7]:
mat = mandelbrot(maxiter,xpixels)
#draw_image(mat)

In [9]:
%timeit -n 1 -r 3 mat=mandelbrot(maxiter,xpixels)

5.74 ms ± 344 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [10]:
# with CUDA
from numba import cuda

In [30]:
# counts the number of iterations until the function diverges or
# returns the iteration threshold that we check until
@cuda.jit
def countIterationsUntilDivergent_GPU(mat,realAxis,realAxisLen,imaginaryAxis,imaginaryAxisLen, threshold):
    x = cuda.blockIdx.x
    y = cuda.threadIdx.x
    if x>=realAxisLen or y>=imaginaryAxisLen: return
    c = complex(realAxis[x], imaginaryAxis[y])
    z = complex(0, 0)
    for iteration in range(threshold):
        z = (z*z) + c

        # if abs(z) > 4:                        # 0.063 sec
        if z.real*z.real+z.imag*z.imag > 4*4:   # 0.017 sec
            break
    
    mat[x][y] = iteration

In [31]:
def mandelbrot_GPU(threshold, density):
    # location and size of the atlas rectangle
    realAxis = np.linspace(-0.22, -0.219, density)
    imaginaryAxis = np.linspace(-0.70, -0.699, density)
    realAxisLen = len(realAxis)
    imaginaryAxisLen = len(imaginaryAxis)
    # print(realAxis.dtype)
    # 2-D array to represent mandelbrot atlas
    atlas = np.empty((realAxisLen, imaginaryAxisLen))

    #print(type(imaginaryAxisLen))
    
    countIterationsUntilDivergent_GPU[realAxisLen,imaginaryAxisLen](atlas,realAxis,realAxisLen,imaginaryAxis,imaginaryAxisLen, threshold)

    # color each point in the atlas depending on the iteration count
    #for ix in prange(realAxisLen):
    #    for iy in range(imaginaryAxisLen):
    #        cx = realAxis[ix]
    #        cy = imaginaryAxis[iy]
    #        c = complex(cx, cy)
    #        atlas[ix, iy] = countIterationsUntilDivergent_GPU(c, threshold)
    
    return atlas

In [32]:
mat_GPU = mandelbrot_GPU(maxiter,xpixels)
#draw_image(mat_GPU)

/home/haasegu/.local/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [43]:
%timeit -n 100 -r 3 mandelbrot_GPU(maxiter,xpixels)

3.7 ms ± 102 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)
